In [34]:
import os
import requests
import boto3
import json
from decimal import Decimal

In [2]:
url = 'https://api.ebird.org/v2/data/obs/US/recent'
key = os.environ['EBIRD_API_KEY']
header = {'X-eBirdApiToken': key}
params = {'back': 1, 'includeProvisional': True}
response = requests.get(url, headers = header, params=params)


In [3]:
response

<Response [200]>

In [5]:
data = response.json()
data

[{'speciesCode': 'bnhcow',
  'comName': 'Brown-headed Cowbird',
  'sciName': 'Molothrus ater',
  'locId': 'L11149270',
  'locName': 'Habor Ridge Dr, Harrison US-OH 39.25857, -84.74789',
  'obsDt': '2024-01-18 11:58',
  'howMany': 2,
  'lat': 39.258575,
  'lng': -84.747889,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': True,
  'subId': 'S159205336'},
 {'speciesCode': 'perfal',
  'comName': 'Peregrine Falcon',
  'sciName': 'Falco peregrinus',
  'locId': 'L3546352',
  'locName': 'Carnegie Mellon University',
  'obsDt': '2024-01-18 11:57',
  'howMany': 1,
  'lat': 40.4427503,
  'lng': -79.9434328,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': False,
  'subId': 'S159204987'},
 {'speciesCode': 'daejun',
  'comName': 'Dark-eyed Junco',
  'sciName': 'Junco hyemalis',
  'locId': 'L12198831',
  'locName': 'Cayman Chemical',
  'obsDt': '2024-01-18 11:56',
  'howMany': 3,
  'lat': 42.22805,
  'lng': -83.722282,
  'obsValid': True,
  'obsReviewed': False,
  'lo

In [28]:
for item in data:
    print(item)

{'speciesCode': 'bnhcow', 'comName': 'Brown-headed Cowbird', 'sciName': 'Molothrus ater', 'locId': 'L11149270', 'locName': 'Habor Ridge Dr, Harrison US-OH 39.25857, -84.74789', 'obsDt': '2024-01-18 11:58', 'howMany': 2, 'lat': 39.258575, 'lng': -84.747889, 'obsValid': True, 'obsReviewed': False, 'locationPrivate': True, 'subId': 'S159205336'}
{'speciesCode': 'perfal', 'comName': 'Peregrine Falcon', 'sciName': 'Falco peregrinus', 'locId': 'L3546352', 'locName': 'Carnegie Mellon University', 'obsDt': '2024-01-18 11:57', 'howMany': 1, 'lat': 40.4427503, 'lng': -79.9434328, 'obsValid': True, 'obsReviewed': False, 'locationPrivate': False, 'subId': 'S159204987'}
{'speciesCode': 'daejun', 'comName': 'Dark-eyed Junco', 'sciName': 'Junco hyemalis', 'locId': 'L12198831', 'locName': 'Cayman Chemical', 'obsDt': '2024-01-18 11:56', 'howMany': 3, 'lat': 42.22805, 'lng': -83.722282, 'obsValid': True, 'obsReviewed': False, 'locationPrivate': True, 'subId': 'S159204948'}
{'speciesCode': 'turvul', 'com

In [39]:
first_5 = data[1:5]
display(first_5)

[{'speciesCode': 'perfal',
  'comName': 'Peregrine Falcon',
  'sciName': 'Falco peregrinus',
  'locId': 'L3546352',
  'locName': 'Carnegie Mellon University',
  'obsDt': '2024-01-18 11:57',
  'howMany': 1,
  'lat': 40.4427503,
  'lng': -79.9434328,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': False,
  'subId': 'S159204987'},
 {'speciesCode': 'daejun',
  'comName': 'Dark-eyed Junco',
  'sciName': 'Junco hyemalis',
  'locId': 'L12198831',
  'locName': 'Cayman Chemical',
  'obsDt': '2024-01-18 11:56',
  'howMany': 3,
  'lat': 42.22805,
  'lng': -83.722282,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': True,
  'subId': 'S159204948'},
 {'speciesCode': 'turvul',
  'comName': 'Turkey Vulture',
  'sciName': 'Cathartes aura',
  'locId': 'L14035235',
  'locName': "Whitley's Crossroads",
  'obsDt': '2024-01-18 11:56',
  'howMany': 3,
  'lat': 36.8728124,
  'lng': -76.77873,
  'obsValid': True,
  'obsReviewed': False,
  'locationPrivate': True,
  'subId': 'S1

In [32]:
dynamodb = boto3.resource('dynamodb')
table_name = 'RecentObservations'
table = dynamodb.Table(table_name)

dynamodb.Table(name='RecentObservations')


In [43]:
for item in first_5:
    table.put_item(
        TableName = table_name,
        Item = {
                'speciesCode': item['speciesCode'],
                'comName': item['comName'],
                'sciName': item['sciName'],
                'locId': item['locId'],
                'locName': item['locName'],
                'obsDt': item['obsDt'],
                'howMany': item['howMany'],
                'lat': str(item['lat']),
                'lng': str(item['lng']),
                'obsValid': item['obsValid'],
                'obsReviewed': item['obsReviewed'],
                'locationPrivate': item['locationPrivate'],
                'subId':  item['subId']
        }
    )

In [ ]:
for item in first_5:
    table.update_item(
        TableName = table_name,
        Key = {'speciesCode': item['speciesCode'], 'obsDt': item['obsDt']}
        Item = {
                'speciesCode': item['speciesCode'],
                'comName': item['comName'],
                'sciName': item['sciName'],
                'locId': item['locId'],
                'locName': item['locName'],
                'obsDt': item['obsDt'],
                'howMany': item['howMany'],
                'lat': str(item['lat']),
                'lng': str(item['lng']),
                'obsValid': item['obsValid'],
                'obsReviewed': item['obsReviewed'],
                'locationPrivate': item['locationPrivate'],
                'subId':  item['subId']
        }
)

In [38]:
response = table.scan()
print(response)

{'Items': [{'locId': 'L14035235', 'locationPrivate': True, 'lng': Decimal('-76.77873'), 'speciesCode': 'horlar', 'sciName': 'Eremophila alpestris', 'obsReviewed': False, 'howMany': Decimal('1'), 'obsValid': True, 'locName': "Whitley's Crossroads", 'obsDt': '2024-01-18 11:56', 'comName': 'Horned Lark', 'subId': 'S159205287', 'lat': Decimal('36.8728124')}, {'locId': 'L3546352', 'locationPrivate': False, 'lng': Decimal('-79.9434328'), 'speciesCode': 'perfal', 'sciName': 'Falco peregrinus', 'obsReviewed': False, 'howMany': Decimal('1'), 'obsValid': True, 'locName': 'Carnegie Mellon University', 'obsDt': '2024-01-18 11:57', 'comName': 'Peregrine Falcon', 'subId': 'S159204987', 'lat': Decimal('40.4427503')}, {'locId': 'L12198831', 'locationPrivate': True, 'lng': Decimal('-83.722282'), 'speciesCode': 'daejun', 'sciName': 'Junco hyemalis', 'obsReviewed': False, 'howMany': Decimal('3'), 'obsValid': True, 'locName': 'Cayman Chemical', 'obsDt': '2024-01-18 11:56', 'comName': 'Dark-eyed Junco', 's